In [94]:
# create a client for the chroma vector store locally
import chromadb
chroma_client = chromadb.Client()
chroma_client.reset()


Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.
Exiting: Cleaning up .chroma directory


True

In [95]:
import os, openai
os.environ["OPENAI_API_TYPE"] = openai.api_type = "azure"
os.environ["OPENAI_API_VERSION"] = openai.api_version = "2022-12-01"
os.environ["OPENAI_API_BASE"] = openai.api_base = "https://openai-endpoint.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = openai.api_key = "f056ead909e54ea0a2fb570e2febad2b"

In [96]:
# define a function that will be used to embed documents when they are entered into the vector store
from chromadb.utils import embedding_functions
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=os.environ["OPENAI_API_KEY"],
                model_name="text-embedding-ada-002"
            )

In [97]:
# we are going to use the directory loader which will leverage a local version of Unstructured-io
from langchain.document_loaders import DirectoryLoader
loader = DirectoryLoader('data/', glob="**/*.pdf")
docs = loader.load()


Exception ignored in: <function DuckDB.__del__ at 0x11dea4b80>
Traceback (most recent call last):
  File "/Users/sean/Documents/code/smith-nephew-aoai-demo/venv/lib/python3.9/site-packages/chromadb/db/duckdb.py", line 346, in __del__
    self._idx.reset()
AttributeError: 'DuckDB' object has no attribute '_idx'


Exiting: Cleaning up .chroma directory


In [98]:
# check all the documents that were loaded
len(docs)

3

In [99]:
#split every document into chunks 
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(docs)

In [100]:
#generate embeddings for each chunk
import time, tqdm
embeddings = []

for text in tqdm.tqdm(texts):
    try:
        response = openai.Embedding.create(
            input=text.page_content,
            engine="text-embedding-ada-002")
        emb = response['data'][0]['embedding']
        embeddings.append(emb)
    except Exception as e:
        time.sleep(8)
        response = openai.Embedding.create(
            input=text.page_content,
            engine="text-embedding-ada-002")
        emb = response['data'][0]['embedding']
        embeddings.append(emb)

100%|██████████| 62/62 [00:14<00:00,  4.39it/s]


In [101]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# initialize the vector store and create a openai embedding function
azure_embeddings = OpenAIEmbeddings(document_model_name="text-embedding-ada-002",query_model_name="text-embedding-ada-002")
vectorstore = Chroma("my_collection", embedding_function=azure_embeddings)

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.
Exiting: Cleaning up .chroma directory


In [102]:
# add the documents to the vector store
vectorstore._collection.add(
    ids= [f"doc_{i}" for i in range(len(texts))],
    documents=[texts[i].page_content for i in range(len(texts))],
    embeddings=embeddings,
    metadatas=[text.metadata for text in texts]
)

In [103]:
from langchain import VectorDBQAWithSourcesChain
from langchain.llms import AzureOpenAI

#initialize our chain

chain = VectorDBQAWithSourcesChain.from_chain_type(llm=AzureOpenAI(deployment_name="davinci003", model_name="text-davinci-003"), chain_type="stuff", vectorstore=vectorstore)


In [104]:
def return_answer(question):
    answer = chain({'question': question}, return_only_outputs=True)
    answer = f"""{answer['answer']}\n\n source: """
    return answer

In [105]:
with gr.Blocks() as demo:
    text = gr.Textbox(show_label=False, placeholder="Enter text and press enter").style(container=False)
    answerbox = gr.Textbox(show_label=False, placeholder="Answer").style(container=False)
    text.submit(return_answer, inputs=[text], outputs=[answerbox])
demo.launch()

Running on local URL:  http://127.0.0.1:7887

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Users/sean/Documents/code/smith-nephew-aoai-demo/venv/lib/python3.9/site-packages/gradio/routes.py", line 384, in run_predict
    output = await app.get_blocks().process_api(
  File "/Users/sean/Documents/code/smith-nephew-aoai-demo/venv/lib/python3.9/site-packages/gradio/blocks.py", line 1024, in process_api
    result = await self.call_function(
  File "/Users/sean/Documents/code/smith-nephew-aoai-demo/venv/lib/python3.9/site-packages/gradio/blocks.py", line 836, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/Users/sean/Documents/code/smith-nephew-aoai-demo/venv/lib/python3.9/site-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/Users/sean/Documents/code/smith-nephew-aoai-demo/venv/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/Users/sean/Documents/code/smith-nep